# Using OpenAI functions
This walkthrough demonstrates how to incorporate OpenAI function-calling API’s in a chain. We’ll go over: 
1. How to use functions to get structured outputs from ChatOpenAI 
2. How to create a generic chain that uses (multiple) functions 
3. How to create a chain that actually executes the chosen function

In [7]:
from typing import Optional

from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_openai_fn_runnable,
    create_structured_output_chain,
    create_structured_output_runnable,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os
# Load environment variables from the .env file
load_dotenv()

api_key = os.getenv("OPEN_AI_KEY")
organization = os.getenv("OPEN_AI_ORG")
# elevenlabs_key = os.getenv("ELEVENLABS_KEY")

llm = ChatOpenAI(
    model="gpt-3.5-turbo-1106",
    temperature=0,
    openai_api_key=api_key,
    openai_organization=organization,
)

## Getting structured outputs
We can take advantage of OpenAI functions to try and force the model to return a particular kind of structured output. We’ll use `create_structured_output_runnable` to create our chain, which takes the desired structured output either as a Pydantic class or as JsonSchema.

### Using Pydantic classes
When passing in Pydantic classes to structure our text, we need to make sure to have a docstring description for the class. It also helps to have descriptions for each of the classes attributes.

In [4]:
# !pip3 install langchain_core

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")

In [8]:
# For better results in OpenAI function-calling API, it is recommended to explicitly pass the latest model.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world class algorithm for extracting information in structured formats.",
        ),
        (
            "human",
            "Use the given format to extract information from the following input: {input}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

runnable = create_structured_output_runnable(Person, llm, prompt)
runnable.invoke({"input": "Sally is 13"})

Person(name='Sally', age=13, fav_food=None)

In [10]:
from typing import Sequence

class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person] = Field(..., description="The people in the text")


runnable = create_structured_output_runnable(People, llm, prompt)
runnable.invoke(
    {
        "input": "Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally."
    }
)

People(people=[Person(name='Sally', age=13, fav_food=None), Person(name='Joey', age=12, fav_food='spinach'), Person(name='Caroline', age=23, fav_food=None)])

### Using JsonSchema
We can also pass in JsonSchema instead of Pydantic classes to specify the desired structure. 
When we do this, our chain will output JSON corresponding to the properties described in the JsonSchema, instead of a Pydantic class.

In [11]:
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "fav_food": {
            "title": "Fav Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age"],
}

In [12]:
runnable = create_structured_output_runnable(json_schema, llm, prompt)
runnable.invoke({"input": "Sally is 13"})

{'name': 'Sally', 'age': 13}

### [Legacy] LLMChain-based approach

In [14]:
chain = create_structured_output_chain(
    Person, 
    llm,
    prompt, 
    verbose=True)

chain.run("Sally is 13")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input: Sally is 13
Human: Tip: Make sure to answer in the correct format

> Finished chain.


Person(name='Sally', age=13, fav_food=None)

## Creating a generic OpenAI functions chain
To create a generic OpenAI functions chain, we can use the create_openai_fn_runnable method. This is the same as create_structured_output_runnable except that instead of taking a single output schema, it takes a sequence of function definitions.

Functions can be passed in as: - dicts conforming to OpenAI functions spec, - Pydantic classes, in which case they should have docstring descriptions of the function they represent and descriptions for each of the parameters, - Python functions, in which case they should have docstring descriptions of the function and args, along with type hints.

### Using Pydantic classes

In [15]:
class RecordPerson(BaseModel):
    """Record some identifying information about a pe."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")


class RecordDog(BaseModel):
    """Record some identifying information about a dog."""

    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")

In [16]:
from langchain.chains.openai_functions import (
    convert_to_openai_function,
    get_openai_output_parser,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class algorithm for recording entities."),
        (
            "human",
            "Make calls to the relevant function to record the entities in the following input: {input}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

openai_functions = [convert_to_openai_function(f) for f in (RecordPerson, RecordDog)]
llm_kwargs = {"functions": openai_functions}
if len(openai_functions) == 1:
    llm_kwargs["function_call"] = {"name": openai_functions[0]["name"]}
output_parser = get_openai_output_parser((RecordPerson, RecordDog))
runnable = prompt | llm.bind(**llm_kwargs) | output_parser

In [17]:
runnable.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})

RecordPerson(name='Harry', age=5, fav_food='chicken')

For convenience we can use the create_openai_fn_runnable method to help build our Runnable

In [18]:
runnable = create_openai_fn_runnable([RecordPerson, RecordDog], llm, prompt)
runnable.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})

RecordPerson(name='Harry', age=0, fav_food=None)

### Using Python functions
We can pass in functions as Pydantic classes, directly as OpenAI function dicts, or Python functions. To pass Python function in directly, we’ll want to make sure our parameters have type hints, we have a docstring, and we use Google Python style docstrings to describe the parameters.

NOTE: To use Python functions, make sure the function arguments are of primitive types (str, float, int, bool) or that they are Pydantic objects.

In [20]:
class OptionalFavFood(BaseModel):
    """Either a food or null."""

    food: Optional[str] = Field(
        None,
        description="Either the name of a food or null. Should be null if the food isn't known.",
    )


def record_person(name: str, age: int, fav_food: OptionalFavFood) -> str:
    """Record some basic identifying information about a person.

    Args:
        name: The person's name.
        age: The person's age in years.
        fav_food: An OptionalFavFood object that either contains the person's favorite food or a null value. Food should be null if it's not known.
    """
    return f"Recording person {name} of age {age} with favorite food {fav_food.food}!"

In [21]:
runnable = create_openai_fn_runnable([record_person], llm, prompt)
runnable.invoke(
    {
        "input": "The most important thing to remember about Tommy, my 12 year old, is that he'll do anything for apple pie."
    }
)

{'name': 'Tommy', 'age': 12, 'fav_food': {'food': 'apple pie'}}

In [22]:
def record_dog(name: str, color: str, fav_food: OptionalFavFood) -> str:
    """Record some basic identifying information about a dog.

    Args:
        name: The dog's name.
        color: The dog's color.
        fav_food: An OptionalFavFood object that either contains the dog's favorite food or a null value. Food should be null if it's not known.
    """
    return f"Recording dog {name} of color {color} with favorite food {fav_food}!"


runnable = create_openai_fn_runnable([record_person, record_dog], llm, prompt)
runnable.invoke(
    {
        "input": "I can't find my dog Henry anywhere, he's a small brown beagle. Could you send a message about him?"
    }
)

{'arguments': {'name': 'Henry', 'color': 'brown', 'fav_food': {'food': None}},
 'name': 'record_dog'}

### [Legacy] LLMChain-based approach

In [23]:
chain = create_openai_fn_chain([RecordPerson, RecordDog], llm, prompt, verbose=True)
chain.run("Harry was a chubby brown beagle who loved chicken")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for recording entities.
Human: Make calls to the relevant function to record the entities in the following input: Harry was a chubby brown beagle who loved chicken
Human: Tip: Make sure to answer in the correct format

> Finished chain.


RecordPerson(name='Harry', age=5, fav_food='chicken')